In [1]:
import numpy as np

class DecisionTreeNode:
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, value=None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

class DecisionTreeClassifierScratch:
    def __init__(self, max_depth=5, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.root = None
        
    def fit(self, X, y):
        self.root = self._build_tree(X,y)
        
    def predict(self, X):
        return np.array([self._predict_row(x, self.root) for x in X])
        
    def _predict_row(self, x, node):
        if node.value is not None:
            return node.value
        if x[node.feature_index]<node.threshold:
            return self._predict_row(x, node.left)
        else:
            return self._predict_row(x, node.right)

    def _build_tree(self, X, y, depth=0):
        num_samples, num_features = X.shape
        num_classes = len(set(y))

        if (depth >= self.max_depth) or (num_samples < self.min_samples_split) or (num_classes == 1):
            leaf_value = self._most_common_label(y)
            return DecisionTreeNode(value=leaf_value)

        best_feat, best_thresh = self._best_split(X, y)

        left_idx = X[:, best_feat] < best_thresh
        right_idx = X[:, best_feat] >= best_thresh
        left = self._build_tree(X[left_idx], y[left_idx], depth + 1)
        right = self._build_tree(X[right_idx], y[right_idx], depth + 1)

        return DecisionTreeNode(best_feat, best_thresh, left, right)
        
    def _best_split(self, X, y):
        best_gini = float("inf")
        best_feat = None
        best_thresh = None 
       
        n_samples, n_features = X.shape

        for feature_index in range(n_features):
            thresholds = np.unique(X[:, feature_index])
            for threshold in thresholds:
                gini = self._gini_split(X, y, feature_index, threshold)
                if gini < best_gini:
                    best_gini = gini
                    best_feat = feature_index
                    best_thresh = threshold
        return best_feat, best_thresh 
    def _gini_split(self, X, y, feature_index, threshold):
        left_mask = X[:, feature_index] < threshold
        right_mask = ~left_mask

        left_y = y[left_mask]
        right_y = y[right_mask]

        left_gini = self._gini_index(left_y)
        right_gini = self._gini_index(right_y)

        n = len(y)
        return (len(left_y) / n) * left_gini + (len(right_y) / n) * right_gini

    def _gini_index(self, y):
        if len(y) == 0:
            return 0
        classes, counts = np.unique(y, return_counts=True)
        probs = counts / len(y)
        return 1 - np.sum(probs ** 2)

    def _most_common_label(self, y):
        return np.bincount(y).argmax()

# **Example Use**

In [2]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

data = load_iris()
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

tree = DecisionTreeClassifierScratch(max_depth=10)
tree.fit(X_train, y_train)

preds = tree.predict(X_test)
accuracy = np.mean(preds == y_test)
print("Accuracy:", accuracy)

Accuracy: 0.9333333333333333
